Create flower Segmentation

In [7]:
import os
from tqdm import tqdm
import numpy as np
from PIL import Image

# Paths
image_dir = "data/jpg"
mask_dir = "data/segmim"
output_dir = "data/segmented"
os.makedirs(output_dir, exist_ok=True)

# Process all images
for i in tqdm(range(1, 8189 + 1)):  # 1-based indexing
    img_name = f"image_{i:05d}.jpg"
    mask_name = f"segmim_{i:05d}.jpg"

    try:
        img = Image.open(os.path.join(image_dir, img_name)).convert("RGB")
        mask = Image.open(os.path.join(mask_dir, mask_name)).convert("L")

        #convert to numpy
        img_np = np.array(img)
        mask_np = np.array(mask)

        mask_bool = mask_np > 128 #convert grayscale to binary (True=white)
        segmented = img_np.copy()
        segmented[~mask_bool] = 0 #set non flower pixels to black

        seg_img = Image.fromarray(segmented)
        seg_img.save(os.path.join(output_dir, f"segmented_{i:05d}.png"))
    except Exception as e:
        print(f"Error processing {img_name}: {e}")


100%|██████████| 8189/8189 [14:46<00:00,  9.24it/s]


Analyze petals

In [16]:
import cv2
import numpy as np
from PIL import Image
import os
import json
from tqdm import tqdm

# Directories
segmented_dir = "data/segmented"
output_img_dir = "data/contours"
output_json_dir = "data/petal_data"
os.makedirs(output_img_dir, exist_ok=True)
os.makedirs(output_json_dir, exist_ok=True)

for i in tqdm(range(1, 8189 + 1)):  # Loop through all images
    img_name = f"segmented_{i:05d}.png"
    
    try:
        # Load segmented image
        img_path = os.path.join(segmented_dir, img_name)
        img = cv2.imread(img_path)
        if img is None:
            raise FileNotFoundError(f"Image not found: {img_path}")
        
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        _, binary = cv2.threshold(gray, 10, 255, cv2.THRESH_BINARY)
        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Store petal metrics
        petal_data = []
        output = img.copy()
        petal_index = 1

        for cnt in contours:
            area = cv2.contourArea(cnt)
            if area > 500:  # Skip small/noisy blobs
                perimeter = cv2.arcLength(cnt, True)
                x, y, w, h = cv2.boundingRect(cnt)
                aspect_ratio = float(w) / h
                circularity = 4 * np.pi * area / (perimeter ** 2 + 1e-5)

                petal_data.append({
                    "id": petal_index,
                    "area": round(area, 2),
                    "aspect_ratio": round(aspect_ratio, 2),
                    "circularity": round(circularity, 2)
                })

                # Draw and label
                cv2.drawContours(output, [cnt], -1, (0, 255, 0), 2)
                M = cv2.moments(cnt)
                if M["m00"] != 0:
                    cx = int(M["m10"] / M["m00"])
                    cy = int(M["m01"] / M["m00"])
                    cv2.putText(output, str(petal_index), (cx, cy), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)

                petal_index += 1

        # Save contour image
        output_img = Image.fromarray(cv2.cvtColor(output, cv2.COLOR_BGR2RGB))
        output_img.save(os.path.join(output_img_dir, f"contours_{i:05d}.png"))

        # Save JSON data
        with open(os.path.join(output_json_dir, f"petals_{i:05d}.json"), "w") as f:
            json.dump(petal_data, f, indent=4)

    except Exception as e:
        print(f"Error processing {img_name}: {e}")


  0%|          | 0/8189 [00:00<?, ?it/s]

100%|██████████| 8189/8189 [09:55<00:00, 13.74it/s]
